This notebook uses Julia 1.7.2 and is self-contained (does not rely on other notebooks).

In [1]:
using JuMP, Gurobi, Plots, DataFrames, CSV, Shapefile

In [2]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-04


Gurobi.Env(Ptr{Nothing} @0x0000000159f82400, false, 0)

In [3]:
# Display all columns by default
ENV["COLUMNS"] = 1000

1000

Average ivermectin/person: 9.27

In [4]:
drc_data = CSV.read("data-CD-Oncho-iu.csv", DataFrame)
drc_data = filter(row -> row."Year" == 2019, drc_data)
drc_data[!, :"ivermectin_amt"] = drc_data."PopReq" .* 9.27

drc_data

,CONTINENT,REGION,WHO_REGION,ADMIN0,ADMIN0ID,ADMIN0_FIP,ADMIN0ISO2,ADMIN0ISO3,ADMIN1,ADMIN1ID,ADMIN2,ADMIN2ID,Alt_ADMIN2,ADMIN3,ADMIN3ID,IUs_ADM,IUs_NAME,IU_ID,IU_CODE,Year,PopTot,PopPreSAC,PopSAC,PopAdult,Endemicity,PopReq,PopTrg,PopTreat,Cov,Cum_MDA,MDA,EffMDA,MDA_n,EffMDA_n,EpiCov,EpiMDA,EpiEffMDA,EpiMDA_n,EpiEffMDA_n,MDA_scheme,Endemicity_ID,ivermectin_amt
,String7,String15,String7,String,Int64,String3,String3,String3,String15,Int64,String15,Int64,String7,String31,Int64,String7,String31,Int64,String15,Int64,Int64,Int64,Int64,Int64,String,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,String15,Int64,Float64
1,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwilu,24753,Bagata,0,null,Bagata,13816,ADM3,Bagata,13816,COD2475313816,2019,203103,34528,71086,93427,Unknown (under LF MDA),0,0,0,0.0,4,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0
2,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwilu,24753,Bandundu,0,null,Bandundu,13818,ADM3,Bandundu,13818,COD2475313818,2019,189523,32219,66333,87181,Unknown (under LF MDA),0,0,0,0.0,4,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0
3,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwango,24752,Kenge,0,null,Boko,13819,ADM3,Boko,13819,COD2475213819,2019,292409,49710,102343,134508,Endemic (under MDA),292409,236851,236868,100.01,6,1,1,4,4,81.01,1,1,4,3,IVM,2,2.71063e6
4,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Mai-Ndombe,24754,Kutu,0,null,Bokoro,13820,ADM3,Bokoro,13820,COD2475413820,2019,259679,44145,90888,119452,Unknown (consider Oncho Elimination Mapping),0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,Not delivered,4,0.0
5,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Mai-Ndombe,24754,Bolobo,0,null,Bolobo,13821,ADM3,Bolobo,13821,COD2475413821,2019,149070,25342,52175,68572,Unknown (consider Oncho Elimination Mapping),0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,Not delivered,4,0.0
6,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Mai-Ndombe,24754,Oshwe,0,null,Bosobe,13822,ADM3,Bosobe,13822,COD2475413822,2019,133039,22617,46564,61198,Unknown (consider Oncho Elimination Mapping),0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,Not delivered,4,0.0
7,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwilu,24753,Bulungu,0,null,Bulungu,13823,ADM3,Bulungu,13823,COD2475313823,2019,315311,53603,110359,145043,Unknown (under LF MDA),0,0,0,0.0,4,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0
8,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwilu,24753,Bulungu,0,null,Djuma,13824,ADM3,Djuma,13824,COD2475313824,2019,248915,42316,87120,114501,Unknown (under LF MDA),0,0,0,0.0,4,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0
9,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwango,24752,Feshi,0,null,Feshi,13825,ADM3,Feshi,13825,COD2475213825,2019,160401,27268,56140,73784,Unknown (under LF MDA),0,0,0,0.0,2,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0


In [5]:
drc_loc_T = Shapefile.Table("cod_admbnda_rgc_itos_20190911_shp/cod_admbndp_admALL_rgc_itos_20190911.shp")
drc_loc = DataFrame(drc_loc_T)

,geometry,ADM0_FR,ADM0_PCODE,ADM1_FR,ADM1_PCODE,ADM2_FR,ADM2_PCODE,ADM2_REF,ADM2ALT1FR,ADM2ALT2FR,POINT_X,POINT_Y
,Point,String,String,String,String,String,String,String?,Missing,Missing,Float64,Float64
1,"Point(30.2382, 3.85842)",République démocratique du Congo (la),CD,Haut-Uele,CD53,Aba,CD5306,missing,missing,missing,30.2382,3.85842
2,"Point(23.7546, 2.98007)",République démocratique du Congo (la),CD,Bas-Uele,CD52,Aketi,CD5204,missing,missing,missing,23.7546,2.98007
3,"Point(26.0726, 4.47187)",République démocratique du Congo (la),CD,Bas-Uele,CD52,Ango,CD5207,missing,missing,missing,26.0726,4.47187
4,"Point(30.707, 3.13686)",République démocratique du Congo (la),CD,Ituri,CD54,Ariwara,CD5410,missing,missing,missing,30.707,3.13686
5,"Point(30.5333, 3.06645)",République démocratique du Congo (la),CD,Ituri,CD54,Aru,CD5409,missing,missing,missing,30.5333,3.06645
6,"Point(26.9969, 0.735506)",République démocratique du Congo (la),CD,Tshopo,CD51,Bafwasende,CD5111,missing,missing,missing,26.9969,0.735506
7,"Point(17.7111, -3.90487)",République démocratique du Congo (la),CD,Kwilu,CD32,Bagata,CD3202,missing,missing,missing,17.7111,-3.90487
8,"Point(25.9729, 3.09299)",République démocratique du Congo (la),CD,Bas-Uele,CD52,Bambesa,CD5209,missing,missing,missing,25.9729,3.09299
9,"Point(25.4582, 1.58373)",République démocratique du Congo (la),CD,Tshopo,CD51,Banalia,CD5110,missing,missing,missing,25.4582,1.58373


In [6]:
drc_loc_only = drc_loc[!, ["ADM2_FR", "POINT_X", "POINT_Y"]]

,ADM2_FR,POINT_X,POINT_Y
,String,Float64,Float64
1,Aba,30.2382,3.85842
2,Aketi,23.7546,2.98007
3,Ango,26.0726,4.47187
4,Ariwara,30.707,3.13686
5,Aru,30.5333,3.06645
6,Bafwasende,26.9969,0.735506
7,Bagata,17.7111,-3.90487
8,Bambesa,25.9729,3.09299
9,Banalia,25.4582,1.58373


In [19]:
# Fix name inconsistencies
drc_data[drc_data.ADMIN2 .== "Katako Kombe", :ADMIN2] .= "Katako-Kombe"
drc_data[drc_data.ADMIN2 .== "Oicha", :ADMIN2] .= "Oïcha"
;

In [8]:
# Data join!
drc_data_combined = leftjoin(drc_data, drc_loc_only, on = "ADMIN2" => "ADM2_FR")

,CONTINENT,REGION,WHO_REGION,ADMIN0,ADMIN0ID,ADMIN0_FIP,ADMIN0ISO2,ADMIN0ISO3,ADMIN1,ADMIN1ID,ADMIN2,ADMIN2ID,Alt_ADMIN2,ADMIN3,ADMIN3ID,IUs_ADM,IUs_NAME,IU_ID,IU_CODE,Year,PopTot,PopPreSAC,PopSAC,PopAdult,Endemicity,PopReq,PopTrg,PopTreat,Cov,Cum_MDA,MDA,EffMDA,MDA_n,EffMDA_n,EpiCov,EpiMDA,EpiEffMDA,EpiMDA_n,EpiEffMDA_n,MDA_scheme,Endemicity_ID,ivermectin_amt,POINT_X,POINT_Y
,String7,String15,String7,String,Int64,String3,String3,String3,String15,Int64,String15,Int64,String7,String31,Int64,String7,String31,Int64,String15,Int64,Int64,Int64,Int64,Int64,String,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,String15,Int64,Float64,Float64?,Float64?
1,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwilu,24753,Bagata,0,null,Bagata,13816,ADM3,Bagata,13816,COD2475313816,2019,203103,34528,71086,93427,Unknown (under LF MDA),0,0,0,0.0,4,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0,17.7111,-3.90487
2,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwilu,24753,Bandundu,0,null,Bandundu,13818,ADM3,Bandundu,13818,COD2475313818,2019,189523,32219,66333,87181,Unknown (under LF MDA),0,0,0,0.0,4,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0,17.4452,-3.31634
3,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwango,24752,Kenge,0,null,Boko,13819,ADM3,Boko,13819,COD2475213819,2019,292409,49710,102343,134508,Endemic (under MDA),292409,236851,236868,100.01,6,1,1,4,4,81.01,1,1,4,3,IVM,2,2.71063e6,17.0216,-5.02334
4,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Mai-Ndombe,24754,Kutu,0,null,Bokoro,13820,ADM3,Bokoro,13820,COD2475413820,2019,259679,44145,90888,119452,Unknown (consider Oncho Elimination Mapping),0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,Not delivered,4,0.0,18.2339,-2.98169
5,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Mai-Ndombe,24754,Bolobo,0,null,Bolobo,13821,ADM3,Bolobo,13821,COD2475413821,2019,149070,25342,52175,68572,Unknown (consider Oncho Elimination Mapping),0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,Not delivered,4,0.0,16.3808,-2.68258
6,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Mai-Ndombe,24754,Oshwe,0,null,Bosobe,13822,ADM3,Bosobe,13822,COD2475413822,2019,133039,22617,46564,61198,Unknown (consider Oncho Elimination Mapping),0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,Not delivered,4,0.0,19.9664,-3.10103
7,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwilu,24753,Bulungu,0,null,Bulungu,13823,ADM3,Bulungu,13823,COD2475313823,2019,315311,53603,110359,145043,Unknown (under LF MDA),0,0,0,0.0,4,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0,18.6919,-4.66163
8,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwilu,24753,Bulungu,0,null,Djuma,13824,ADM3,Djuma,13824,COD2475313824,2019,248915,42316,87120,114501,Unknown (under LF MDA),0,0,0,0.0,4,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0,18.6919,-4.66163
9,Africa,Middle Africa,AFRO,Democratic Republic of the Congo,14,CG,CD,COD,Kwango,24752,Feshi,0,null,Feshi,13825,ADM3,Feshi,13825,COD2475213825,2019,160401,27268,56140,73784,Unknown (under LF MDA),0,0,0,0.0,2,0,0,1,1,0.0,0,0,1,1,Not delivered,5,0.0,18.3199,-6.29713


In [9]:
# Look for missing data
filter(row -> ismissing(row."POINT_X"), drc_data_combined)

,CONTINENT,REGION,WHO_REGION,ADMIN0,ADMIN0ID,ADMIN0_FIP,ADMIN0ISO2,ADMIN0ISO3,ADMIN1,ADMIN1ID,ADMIN2,ADMIN2ID,Alt_ADMIN2,ADMIN3,ADMIN3ID,IUs_ADM,IUs_NAME,IU_ID,IU_CODE,Year,PopTot,PopPreSAC,PopSAC,PopAdult,Endemicity,PopReq,PopTrg,PopTreat,Cov,Cum_MDA,MDA,EffMDA,MDA_n,EffMDA_n,EpiCov,EpiMDA,EpiEffMDA,EpiMDA_n,EpiEffMDA_n,MDA_scheme,Endemicity_ID,ivermectin_amt,POINT_X,POINT_Y
,String7,String15,String7,String,Int64,String3,String3,String3,String15,Int64,String15,Int64,String7,String31,Int64,String7,String31,Int64,String15,Int64,Int64,Int64,Int64,Int64,String,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,String15,Int64,Float64,Float64?,Float64?


In [10]:
# Used this to find some naming inconsistencies between the two dataframes
# filter(name -> startswith(name, "O"), drc_loc_only."ADM2_FR")

In [11]:
# Back to regular column width
ENV["COLUMNS"] = 80

80

In [12]:
# May consider changing to great circle distance
function distance(x1, y1, x2, y2)
    sqrt((x1 - x2)^2 + (y1 - y2)^2)
end

distance (generic function with 1 method)

In [20]:
drc_data_combined = drc_data_combined[1:20, :]
;

In [14]:
num_counties = length(drc_data_combined."ADMIN2")
num_vehicles = 5
max_capacity = 1e8 # some viable constraint on the max number of mg that one vehicle can carry
batch_amt = 5e8

drc_loc_x = drc_data_combined[!, "POINT_X"]
drc_loc_y = drc_data_combined[!, "POINT_Y"]
drc_dists = [distance(drc_loc_x[i], drc_loc_y[i], drc_loc_x[j], drc_loc_y[j])
    for i=1:num_counties, j=1:num_counties]

drc_dosage = drc_data_combined[!, "ivermectin_amt"]
;

In [25]:
m = Model(() -> Gurobi.Optimizer(GRB_ENV))

# variables

# path from i->j in service for vehicle n
@variable(m, s_ij[i=1:num_counties, j=1:num_counties], Bin)

# vehicle n services location i
@variable(m, v_in[i=1:num_counties, n=1:num_vehicles], Bin)

# location i is our chosen central hub
@variable(m, c_i[i=1:num_counties], Bin)

# amount of ivermectin delivered to location i by vehicle n
@variable(m, d_in[i=1:num_counties, n=1:num_vehicles] >= 0)
;

In [26]:
# constraints

# vehicle doesn't explode
@constraint(m, [n=1:num_vehicles], sum(v_in[:, n] .* drc_dosage) <= max_capacity)

# nodes in service graph have outdegree <= indegree, except for central hub
@constraint(m, [i=1:num_counties], sum(s_ij[i, :]) - sum(s_ij[:, i]) <= c_i[i] * num_vehicles)

# if i->j and vehicle n serviced location j, then vehicle n serviced location i
@constraint(m, [i=1:num_counties, j=1:num_counties, n=1:num_vehicles],
    v_in[i, n] >= s_ij[i, j] * v_in[j, n])

# all vehicles visit the central hub
@constraint(m, [i=1:num_counties, n=1:num_vehicles], v_in[i, n] >= c_i[i])

# no two vehicles visit the same location
@constraint(m, [i=1:num_counties, n=1:num_vehicles, n2=1:num_vehicles; n2 != n],
    v_in[i, n] + v_in[i, n2] <= 1)

# ivermectin is only delivered to location i if vehicle n visits it
@constraint(m, [i=1:num_counties, n=1:num_vehicles], d_in[i, n] <= v_in[i, n] * max_capacity)

# total ivermectin delivered to any location is at most the amount required
@constraint(m, [i=1:num_counties], sum(d_in[i, :]) <= drc_dosage[i])

# all ivermectin should be delivered
@constraint(m, sum(d_in) == batch_amt)
;

In [27]:
# objective

# minimize total travel distance
@objective(m, Min, sum(s_ij[:, :] .* drc_dists))
;

In [28]:
optimize!(m)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 646 rows, 620 columns and 2200 nonzeros
Model fingerprint: 0x35e698f0
Model has 2000 quadratic constraints
Variable types: 100 continuous, 520 integer (520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [4e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+08]
Presolve removed 5 rows and 80 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

User-callback calls 47, time in user-callback 0.00 sec
